In [1]:
import os
import gzip
import shutil
import fp_utils
import requests
import pandas as pd
from pathlib import Path
from bs4 import BeautifulSoup

In [2]:
fp_utils.settings.init_fp_utils()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
url = "https://ftp.ncbi.nlm.nih.gov/pubchem/Compound/CURRENT-Full/SDF/"

In [4]:
dir_to_download = Path('../data/pubchem/sdfs')
dir_to_download.mkdir(parents=True, exist_ok=True)

In [5]:
processed_files = Path("../data/pubchem/processed.txt")
with processed_files.open('a'):
    pass

In [6]:
with open(processed_files, 'r') as f:
    processed_before = set(f.read().split())

In [7]:
processed_before

set()

In [8]:
def get_urls(url, ext):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, features='lxml')
    all_links = [link.get("href") for link in soup("a")]
    return pd.Series(filter(lambda x: x.endswith(ext), all_links))

In [9]:
sdfs = get_urls(url, ".sdf.gz")

In [10]:
file_names = pd.Series(list(sorted(sdfs)))

In [11]:
def download_zips(file_name):
    if file_name in processed_before:
        print("skipped", file_name)
        return
    print("start", file_name)
    with requests.get(url + file_name, stream=True) as r:
        with open(dir_to_download / file_name, 'wb') as f:
            shutil.copyfileobj(r.raw, f)
    with gzip.open(dir_to_download / file_name, 'rb') as f_in:
        with open(dir_to_download / file_name[:-3], 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    os.remove(dir_to_download / file_name)
    with open(processed_files, 'a') as f:
        f.write(str(file_name) + '\n')
    print("finish", file_name)

In [ ]:
file_names.parallel_apply(download_zips)

startstartstartstart Compound_083000001_083500000.sdf.gz 
Compound_000000001_000500000.sdf.gz
  Compound_041500001_042000000.sdf.gzCompound_124000001_124500000.sdf.gz

finish Compound_041500001_042000000.sdf.gz
start Compound_042000001_042500000.sdf.gz
